In [5]:
import tensorflow as tf
import numpy as np
import importlib as imp
# tf.enable_eager_execution() # makes code super slow!
# # Meta-training
# $ python main.py \
#   --gpu_id 0 \
#   --savedir "./results/cifar/taml" --id_dataset 'cifar' --ood_dataset 'svhn' \
#   --mode 'meta_train' --metabatch 4 --n_steps 5 --way 5 --max_shot 50 --query 15 \
#   --n_train_iters 50000 --meta_lr 1e-3 \
#   --alpha_on --omega_on --gamma_on --z_on
# Meta-testing
# $ python main.py \
#   --gpu_id 0 \
#   --savedir "./results/cifar/taml" --id_dataset 'cifar' --ood_dataset 'svhn' \
#   --mode 'meta_test' --metabatch 4 --n_steps 10 --way 5 --max_shot 50 --query 15 \
#   --n_test_episodes 1000 \
#   --alpha_on --omega_on --gamma_on --z_on --n_mc_samples 10
class Arg():
    def __init__(self):
        self.gpu_id = 0
        self.savedir = "./results/cifar/taml"
        self.id_dataset = ['cifar']
        self.ood_dataset = ['svhn']
        self.mode = 'meta_train'
        self.metabatch = 4
        self.n_steps = 5
        self.way = 5
        self.max_shot = 3
        self.query = 2
        self.n_train_iters = 1 # actual: 50000
        self.n_train_iters = 1
        self.meta_lr = 1e-3
        self.alpha_on = True
        self.omega_on = True
        self.gamma_on = True
        self.z_on = True
        self.n_mc_samples = 10
        self.inner_lr = 0.5
args = Arg()

In [1]:
import encoder_tf
imp.reload(encoder_tf)
encoder = encoder_tf.InferenceNetwork(args)
x_train = tf.constant(np.ones(shape=(10, 32, 32, 3)).astype('float64'))
y_train = tf.constant(np.ones(shape=(10, 1)))
x_test  = tf.constant(np.ones(shape=(15, 32, 32, 3)))
y_test  = tf.constant(np.ones(shape=(15, 1)))
encoder.forward(inputs=(x_train, y_train), sample=True, reuse=tf.AUTO_REUSE)

/home/jason/anaconda3/envs/learningtobalance/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jason/anaconda3/envs/learningtobalance/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jason/anaconda3/envs/learningtobalance/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/jason/anac

NameError: name 'imp' is not defined

In [3]:
import data_tf
imp.reload(data_tf)
from collections import OrderedDict
# id_data =   OrderedDict([(data, data_tf.Data(data)) for data in args.id_dataset])
d = data_tf.Data(args.id_dataset[0])
xtr_all, ytr_all, xte_all, yte_all = d.generate_episode(args, split='mtr', n_episodes=3, mode=1)
print(xtr_all.shape)
print(ytr_all.shape)

classes:  [18 19 13 54 41]
y_tr dimension for current class:  (3, 5)
y_tr dimension for current class:  (2, 5)
y_tr dimension for current class:  (2, 5)
y_tr dimension for current class:  (2, 5)
y_tr dimension for current class:  (3, 5)
classes:  [61 50 25  4 27]
y_tr dimension for current class:  (2, 5)
y_tr dimension for current class:  (2, 5)
y_tr dimension for current class:  (2, 5)
y_tr dimension for current class:  (2, 5)
y_tr dimension for current class:  (2, 5)
classes:  [31  5 33 36 58]
y_tr dimension for current class:  (2, 5)
y_tr dimension for current class:  (2, 5)
y_tr dimension for current class:  (2, 5)
y_tr dimension for current class:  (2, 5)
y_tr dimension for current class:  (2, 5)
(3, 12, 3072)
(3, 12, 5)


In [7]:
import encoder_tf
imp.reload(encoder_tf)
encoder = encoder_tf.InferenceNetwork(args)
omega, gamma, zeta, KL = encoder.forward(inputs=(xtr_all[0], ytr_all[0]), sample=True, reuse=tf.AUTO_REUSE)

|||encoder: get_posterior()
x shape:  (12, 3072)
y shape:  (12, 5)
<<<<encoder 1>>>
x reshaped before convolution:  (12, 32, 32, 3)
x after convolution 1:  (12, 32, 32, 10)
x after max pooling 1:  (12, 16, 16, 10)
x after convolution 2:  (12, 16, 16, 10)
x after max pooling 2:  (12, 8, 8, 10)
x after flatten:  (12, 640)
x after dense layer:  (12, 64)
<<<statistics pooling 1>>>
y_sum: (sum reduced along dimension 1) Tensor("Sum_18:0", shape=(12,), dtype=float64)
y_ (argmax of y on dimension 1) :  Tensor("ArgMax_2:0", shape=(12,), dtype=int64)
iterate over number of classes 5
	N_c:  ()
	x_c shape:  (?, 64)
|||encoder: _statistics_pooling
	s_c (from statistics pooling x_c:  Tensor("stack_14:0", shape=(64, 3), dtype=float64)
	N_c:  ()
	x_c shape:  (?, 64)
|||encoder: _statistics_pooling
	s_c (from statistics pooling x_c:  Tensor("stack_15:0", shape=(64, 3), dtype=float64)
	N_c:  ()
	x_c shape:  (?, 64)
|||encoder: _statistics_pooling
	s_c (from statistics pooling x_c:  Tensor("stack_16:0",

In [10]:
print("Omega, shape {omega.shape}")
print("Gamma, a dictionary:")
for parameter_name in gamma.keys():
    print("\t", {parameter_name}, "shape:", gamma[parameter_name].shape)
print("Zeta, a dictionary:")
for parameter_name in zeta.keys():
    print("\t", {parameter_name}, "shape:", zeta[parameter_name].shape)
print("KL: ", KL)

Omega, shape {omega.shape}
Gamma, a dictionary:
	 {'conv2_b'} shape: (1,)
	 {'conv1_b'} shape: (1,)
	 {'conv1_w'} shape: (1,)
	 {'conv3_b'} shape: (1,)
	 {'conv2_w'} shape: (1,)
	 {'conv4_w'} shape: (1,)
	 {'conv3_w'} shape: (1,)
	 {'conv4_b'} shape: (1,)
	 {'dense_b'} shape: (1,)
	 {'dense_w'} shape: (1,)
Zeta, a dictionary:
	 {'conv2_b'} shape: (32,)
	 {'conv1_b'} shape: (32,)
	 {'conv1_w'} shape: (32,)
	 {'conv3_b'} shape: (32,)
	 {'conv2_w'} shape: (32,)
	 {'conv4_w'} shape: (32,)
	 {'conv3_w'} shape: (32,)
	 {'conv4_b'} shape: (32,)
KL:  Tensor("add_7:0", shape=(), dtype=float64)


In [6]:
from __future__ import print_function
import time
import os
from collections import OrderedDict

import model_tf
imp.reload(model_tf)
import data_tf
imp.reload(data_tf)
from accumulator_tf import Accumulator
from misc_tf import get_train_op, print_balancing_variables, str2list

# clear varaibles
tf.reset_default_graph()

os.environ['CUDA_VISIBLE_DEVICES'] = str(args.gpu_id)

if not os.path.isdir(args.savedir):
  os.makedirs(args.savedir)

# for generating episode
id_data =   OrderedDict([(data, data_tf.Data(data)) for data in args.id_dataset])
ood_data =  OrderedDict([(data, data_tf.Data(data)) for data in args.ood_dataset])

# model object
model = model_tf.LearningToBalance(args)
placeholders = model.get_placeholders()

# with sampling (for meta-training)
net = model.forward_outer_multiple(sample=True, reuse=False)
net_cent = net['cent']
net_kl = net['kl']
net_acc = net['acc']
net_acc_mean = tf.reduce_mean(net['acc'])

# without sampling (for meta-validation)
tnet = model.forward_outer_multiple(sample=False, reuse=True)
tnet_cent = tnet['cent']
tnet_kl = tnet['kl']
tnet_acc = tnet['acc']
tnet_acc_mean = tf.reduce_mean(tnet['acc'])
tnet_weights = tnet['weights']

def meta_train():
  global_step = tf.train.get_or_create_global_step()
  optim = tf.train.AdamOptimizer(args.meta_lr)
  outer_loss = net_cent + net_kl
  meta_train_op = get_train_op(optim, outer_loss, clip=[-3., 3.],
      global_step=global_step)

  saver = tf.train.Saver(tf.trainable_variables(), max_to_keep=10)
  logfile = open(os.path.join(args.savedir, 'train.log'), 'w')

  # flush out the arguments
  argdict = vars(args)
  print(argdict)
  for k, v in argdict.items():
    logfile.write(k + ': ' + str(v) + '\n')
  logfile.write('\n')

  config = tf.ConfigProto()
  config.gpu_options.allow_growth = True
  sess = tf.Session(config=config)
  sess.run(tf.global_variables_initializer())

  meta_train_logger = Accumulator('cent', 'kl', 'id_acc')
  meta_train_to_run = [meta_train_op, net_cent, net_kl, net_acc_mean]

  # multi-dataset meta-validation loggers
  if len(args.id_dataset) > 1:
    meta_val_logger = OrderedDict([(data_name, Accumulator('cent', 'kl', 'id_acc'))
      for data_name in args.id_dataset + ['val']])
  # single dataset meta-validation logger
  else:
    meta_val_logger = { 'val': Accumulator('cent', 'kl', 'id_acc', 'ood_acc') }
  id_run = [tnet_cent, tnet_kl, tnet_acc_mean]
  ood_run = tnet_acc_mean

  for i in range(1, args.n_train_iters+1):
    # feed_dict
    data_name = args.id_dataset[i%len(args.id_dataset)]
    episode = id_data[data_name].generate_episode(
        args, split='mtr', n_episodes=args.metabatch)
    fd_mtr = dict(zip(placeholders, episode))
    meta_train_logger.accum(
        sess.run(meta_train_to_run, feed_dict=fd_mtr))

    if i % args.save_freq == 0:
      saver.save(sess, os.path.join(args.savedir, 'model-{}'.format(i)))

    if i % 100 == 0:
      line = 'Iter %d start, learning rate %f' % (i, args.meta_lr)
      print('\n' + line)
      logfile.write('\n' + line + '\n')
      meta_train_logger.print_(header='train', logfile=logfile)
      meta_train_logger.clear()

    # meta-validation
    if i % 10000 == 0:
      for j in range(3000 // args.metabatch):
        # valdate on in-distribution (ID) dataset(s)
        for data_name in args.id_dataset:
          id_episode = id_data[data_name].generate_episode(
              args, split='mval', n_episodes=args.metabatch)
          id_cent, id_kl, id_acc = sess.run(id_run,
              feed_dict=dict(zip(placeholders, id_episode)))

          if len(args.id_dataset) > 1:
            meta_val_logger[data_name].accum([id_cent, id_kl, id_acc])
            meta_val_logger['val'].accum([id_cent, id_kl, id_acc])

        # valiate on out-of-distribution (OOD) dataset
        if args.ood_dataset[0] in ['svhn', 'cub']:
          ood_episode = ood_data[args.ood_dataset[0]].generate_episode(
              args, split='mval', n_episodes=args.metabatch)
          ood_acc = sess.run(ood_run,
              feed_dict=dict(zip(placeholders, ood_episode)))
          meta_val_logger['val'].accum([id_cent, id_kl, id_acc, ood_acc])

      for data_name, logger in meta_val_logger.items():
          logger.print_(header='%s  '%data_name, logfile=logfile)
          logger.clear()

    # print balancing variables (omega, gamma) with 10 random tasks
    if args.gamma_on or args.omega_on:
      ntask = 10
      if i % 1000 == 0:
        testlist = [('ID ', id_data[args.id_dataset[0]])]
        if len(args.id_dataset) == 1:
          testlist.append(('OOD', ood_data[args.ood_dataset[0]]))
        for flag, data in testlist:
          # episode
          episode = data.generate_episode(
              args, split='mval', n_episodes=ntask)
          omega, gamma = sess.run(
              model.get_balancing_variables(ntask=ntask),
              feed_dict=dict(zip(placeholders, episode)))

          print_balancing_variables(
              args, flag, episode, ntask, omega, gamma, logfile)

  logfile.close()

def meta_test():
  # choose the best model
  f = open(os.path.join(args.savedir, 'train.log'), 'r')
  lines = [l.split(' ') for l in f.readlines()]
  acc_id, acc_ood, idx_list = [], [], []
  for i, l in enumerate(lines):
    if l[0] == 'val':
      if len(args.id_dataset) > 1:
        idx_list.append(int(lines[i-5][1]))
        acc_id.append(float(l[-1]))
      else:
        idx_list.append(int(lines[i-2][1]))
        acc_id.append(float(l[-3][:-1]))
        acc_ood.append(float(l[-1]))
  acc = np.array(acc_id) if len(args.id_dataset) > 1 \
          else np.array(acc_id) + np.array(acc_ood)
  best_idx = idx_list[np.argmax(acc)]

  config = tf.ConfigProto()
  config.gpu_options.allow_growth = True
  sess = tf.Session(config=config)
  saver = tf.train.Saver(tnet_weights, max_to_keep=10)
  saver.restore(sess, os.path.join(args.savedir, 'model-%d' % best_idx))

  logfile = open(os.path.join(args.savedir,
    'test_step%d_mc%d.log' % (args.n_steps, args.n_mc_samples)), 'w')
  start = time.time()

  line = '\nMeta-test model-%d\n' % best_idx
  print(line)
  logfile.write(line + '\n')

  if args.n_mc_samples > 0:
    to_run = model.forward_outer_multiple_repeat(n_sample=args.n_mc_samples)
  else:
    to_run = tnet_acc

  for flag, datalist in [('id ', id_data), ('ood', ood_data)]:
    for data_name, data in datalist.items():
      acc = []
      for j in range(args.n_test_episodes // args.metabatch):
        placeholders = model.get_placeholders()
        episode = data.generate_episode(
            args, split='mte', n_episodes=args.metabatch)
        acc.append(sess.run(
          to_run, feed_dict=dict(zip(placeholders, episode))))
        if (j+1)*args.metabatch % 100 == 0:
          print('(%.3f secs) test episode %d done' \
              % (time.time()-start, (j+1)*args.metabatch ))

      acc = 100.*np.concatenate(acc, axis=0)
      acc_mean = np.mean(acc)
      acc_95conf = 1.96*np.std(acc)/float(np.sqrt(args.n_test_episodes))
      result = '%s/%s acc: %.2f +- %.2f\n'%(flag, data_name, acc_mean, acc_95conf)
      print(result)
      logfile.write(result)

  logfile.close()

if args.mode == 'meta_train':
    meta_train()
if args.mode == 'meta_test':
    meta_test()

xtr: <class 'tensorflow.python.framework.tensor_shape.TensorShape'>
xtr: <class 'tensorflow.python.framework.tensor_shape.TensorShape'>


KeyboardInterrupt: 